In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.image import ContainerImage
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig,Model

In [ ]:
ws = Workspace.from_config()

In [ ]:
webservice_name = 'turbofan-rul'

run = [x for x in ws.experiments['gbr-turbofan'].get_runs()][0]
run_metrics = run.get_metrics()

print(run)

model = run.register_model(model_name = "turbofan-rul", model_path = "outputs/model.pkl", tags={"mae":run_metrics["mae"],"python version": sys.version[0:6]})

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,auth_enabled=True)

In [ ]:
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=Environment.get(workspace=ws,name="sklearn 2.2"))
                               
try:
    service = Webservice(Webservice, webservice_name)        
    service.delete()    
except Exception as e:
    print("No Existing Service")

service = Model.deploy(ws, webservice_name, [model], inference_config, deployment_config)

#service.get_logs()
service.wait_for_deployment(show_output = True)

#print(service.get_keys())
print(service.scoring_uri)